In [4]:
from pathlib import Path
import json

In [5]:
with open('../topics.json', 'r') as file:
    topic = json.load(file)

In [6]:
# path = './retrieve/gpt-3.5/train/1/'
# for t in topic.keys():
#     sub = path + t
#     sub = Path(sub)
#     sub.mkdir()
#     sub = Path(path + t + '/true')
#     sub.mkdir()
#     sub = Path(path + t + '/false')
#     sub.mkdir()
#     sub = Path(path + t + '/mostly_true')
#     sub.mkdir()
#     sub = Path(path + t + '/mostly_false')
#     sub.mkdir()
#     sub = Path(path + t + '/half_true')
#     sub.mkdir()
#     sub = Path(path + t + '/on_fire')
#     sub.mkdir()

In [7]:
label = ['true', 'false', 'mostly_true', 'mostly_false', 'half_true', 'on_fire']

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from fastchat.model import load_model, get_conversation_template, add_model_args

model_path='lmsys/vicuna-7b-v1.5'
revision='main'
device='cuda'
gpus=None
num_gpus=1
max_gpu_memory=None
load_8bit=False
cpu_offloading=False
gptq_ckpt=None
gptq_wbits=16
gptq_groupsize=-1
gptq_act_order=False
awq_ckpt=None
awq_wbits=16
awq_groupsize=-1
temperature=0.7
repetition_penalty=1.0
max_new_tokens=512
debug=False

model, tokenizer = load_model(
    model_path,
    device=device,
    num_gpus=num_gpus,
    max_gpu_memory=max_gpu_memory,
    load_8bit=load_8bit,
    cpu_offloading=cpu_offloading,
    revision=revision,
    debug=debug,
)

/home/stu_109550068/.conda/envs/wy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.78s/it]


In [9]:
@torch.inference_mode()
def main(msg):
    
    conv = get_conversation_template(model_path)
    conv.append_message(conv.roles[0], msg)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    inputs = tokenizer([prompt])
    inputs = {k: torch.tensor(v).to(device) for k, v in inputs.items()}
    output_ids = model.generate(
        **inputs,
        do_sample=True if temperature > 1e-5 else False,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        max_new_tokens=max_new_tokens,
    )

    if model.config.is_encoder_decoder:
        output_ids = output_ids[0]
    else:
        output_ids = output_ids[0][len(inputs["input_ids"][0]) :]
    outputs = tokenizer.decode(
        output_ids, skip_special_tokens=True, spaces_between_special_tokens=False
    )

    # print(f"{conv.roles[0]}: {msg}")
    # print(f"{conv.roles[1]}: {outputs}")
    return outputs

In [ ]:
for t in ['Health Care']:
    # educate, economic...
    for l in ['false']:
        path = Path(f'../dataset/test/{t}/{l}/')
        num = len(list(path.glob('*')))
        for i in range(num):
            idx = i + 1
            with open(f'../dataset/test/{t}/{l}/{idx}/info.json', 'r') as file:
                info = json.load(file)
            claim = info['claim']
            claimant = info['claimant']
            date = info['claim_date']
            
            # path = Path(f'../dataset/test/{t}/{l}/{idx}/premise/')
            # sub = path.glob('*')
            # output = []
            # for article in sub:
            #     with open(article, 'r') as file:
            #         data = json.load(file)
            #         with torch.cuda.device('cuda'):
            #             output.append(main(f"""Output a no more than 50-words argument toward the claim by reading the reference information.\nClaim: {claim}\nClaimant: {claimant}\nClaim date: {date}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}\nNote: If the provided reference is not related to the claim, directly output not related."""))
            # with open(f'./no_retrieve/vicuna/test/{t}/{l}/{idx}.json', 'w') as file:
            #     json.dump(output, file) 
            
            for group in range(1):
                path = Path(f'../retrieval/{group+1}/test/{t}/{l}/{idx}/')
                sub = path.glob('*')
                output = []
                for article in sub:
                    with open(article, 'r') as file:
                        data = json.load(file)
                        with torch.cuda.device('cuda'):
                            response = main(f"""Output a no more than 50-words argument evaluating the authenticity of a claim by reading a reference.\nClaim: {claim}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}\nNote: If the provided reference is not related to the claim, directly output not related.""")
                            if "Not related" in response or "not related" in response:
                                output.append(response)
                                continue
                            refinement = main(f"""Evaluate whether the argument is aligned with the information and evidence in the reference, don't consider the claim, output aligned/not aligned, and provide the reasons.\nArgument: {response}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}""")
                            print(response)
                            print(refinement)
                            while "not aligned" in refinement or "Not aligned" in refinement:
                                response = main(f"""Previous argument fails to align with the reference, check the feedback and output a no more than 50-words argument evaluating the authenticity of a claim again.\nFeedback: {refinement}\nClaim: {claim}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}\nNote: If the provided reference is not related to the claim, directly output not related.""")
                                if "Not related" in response or "not related" in response:
                                    output.append(response)
                                    continue
                                refinement = main(f"""Evaluate whether the argument is aligned with the information and evidence in the reference, don't consider the claim, output aligned/not aligned, and provide the reasons.\nArgument: {response}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}""")
                                print(response)
                                print(refinement)
                            if "aligned" in refinement or "Aligned" in refinement:
                                output.append(response)
                            else:
                                print(f"ERROR: {refinement}")

In [8]:
for t in topic.keys():
    # educate, economic...
    for l in label:
        path = Path(f'../dataset/test/{t}/{l}/')
        num = len(list(path.glob('*')))
        for i in range(num):
            idx = i + 1
            with open(f'../dataset/test/{t}/{l}/{idx}/info.json', 'r') as file:
                info = json.load(file)
            claim = info['claim']
            claimant = info['claimant']
            date = info['claim_date']
            
            # path = Path(f'../dataset/test/{t}/{l}/{idx}/premise/')
            # sub = path.glob('*')
            # output = []
            # for article in sub:
            #     with open(article, 'r') as file:
            #         data = json.load(file)
            #         with torch.cuda.device('cuda'):
            #             output.append(main(f"""Output a no more than 50-words argument toward the claim by reading the reference information.\nClaim: {claim}\nClaimant: {claimant}\nClaim date: {date}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}\nNote: If the provided reference is not related to the claim, directly output not related."""))
            # with open(f'./no_retrieve/vicuna/test/{t}/{l}/{idx}.json', 'w') as file:
            #     json.dump(output, file) 
            
            for group in range(5):
                path = Path(f'../retrieval/{group+1}/test/{t}/{l}/{idx}/')
                sub = path.glob('*')
                output = []
                for article in sub:
                    with open(article, 'r') as file:
                        data = json.load(file)
                        with torch.cuda.device('cuda'):
                            output.append(main(f"""Output a no more than 50-words argument evaluating the authenticity of a claim by reading a reference.\nClaim: {claim}\nReference: {data if len(str(data))<3000 else str(data)[:3000]}\nNote: If the provided reference is not related to the claim, directly output not related."""))
                with open(f'./retrieve/vicuna/test/{group+1}/{t}/{l}/{idx}.json', 'w') as file:
                    json.dump(output, file) 

In [9]:
# Set your API key
key = "{use your key here}"

In [2]:
data = [
    "Rigorous nonprofit news for Vermont.",
    "* Teachers, high-risk Vermonters eligible for vaccine",
    "* 'Virus in Vermont' series",
    "Governor Scott: Mr. President, Madam Speaker, Mr. Chief Justice, members",
    "of the General Assembly, distinguished guests and fellow Vermonters:",
    "dollar we spend comes directly from hardworking Vermonters, many",
    "growing our economy, making Vermont more affordable and protecting the",
    "To put this in perspective, every county except Chittenden has seen a",
    "decline in the number of people working. Every county, including",
    "Chittenden, has fewer people available for work.",
    "and expand, harder for communities to grow and leaves fewer of us to cover",
    "As our working-age population continues to decline, we simply need more",
    "provide training for Vermonters so they can get a good job and attract new",
    "Put simply, the day-to-day operational costs of state government and",
    "our communities and supports economic growth in all corners of the state.",
    "homes\u2014we must also protect Vermonters online.",
    "Combined with our quality of life, good jobs and working to make Vermont",
    "place we are isn\u2019t enough anymore. If it was, we\u2019d be seeing our labor",
    "working families.",
    "$3.2 million more to Vermont State Colleges.",
    "With this funding, the State Colleges will stop a planned three percent",
    "tuition increase on Vermonters for this coming school year.",
    "delivered in at least three tech centers throughout the state, where",
    "growth by exempting them from Act 250.",
    "areas and give our rural economy a fighting chance.",
    "trail networks. Let\u2019s simplify construction approval in rural industrial",
    "parks and reduce their fees. Let\u2019s help farms create new revenue from",
    "transportation dollars by exempting projects that have environmental",
    "Currently, 75 percent of the state has high-speed broadband or better.",
    "Many communities are trying to expand infrastructure; while there are",
    "To help, and to support our focus on the rural economy, I\u2019m proposing to",
    "approximately $830 million was spent on gasoline sales in Vermont. If this",
    "significantly less, saving drivers more than $500 million.",
    "We need 10 percent\u2014about 50,000\u2014of the cars and trucks on our roads to be",
    "Today, we only have about 1,000 EVs on the road\u2014less than one percent of",
    "We all know transportation costs are high for rural Vermonters\u2014that\u2019s one",
    "But in its final report, the Vermont Climate Action Commission recommends",
    "just about making our state cleaner and healthier\u2014it\u2019s also about growing",
    "been about $19.5 million, and the lowest year was 2015 at $9.9 million.",
    "line with other states, the estate tax is a factor in retirees leaving.",
    "Maine and New York, and more closely with the federal exemption.",
    "Here\u2019s why this is important: currently, we\u2019re one of only 12 states that",
    "Vermonters impacted by this tax are well-advised from tax professionals,",
    "taxpayers here and support a legacy of clean, healthy lakes, rivers and",
    "Last year, we worked together to make sure that low- and middle-income",
    "Vermonters weren\u2019t taxed on their social security, helping retirees live a",
    "Only seven states, including us, fully tax military retirement benefits.",
    "Before we leave this spring, let\u2019s reduce that to six, and eliminate it",
    "Across the state, there are abandoned, vacant and blighted properties",
    "neighborhoods, create more capacity, and increase grand lists as low-value",
    "percent of our housing stock is at least 40 years old, and nearly half of",
    "Finally, so many of the neighborhoods affected by these issues are in and",
    "We also know young families and workers want to be close to jobs, services",
    "exist, in the places we want to see grow.",
    "Attracting and retaining more workers is also a key goal of the paid",
    "plans for all employers and employees in both states.",
    "That\u2019s why our proposal places all eligible state employees from both",
    "government and balances the value of paid family leave with our ability to",
    "Let\u2019s show Vermonters we can work together to reach a goal we agree on,",
    "In every corner of our state, there are communities, businesses and",
    "because we need more working-age people.",
    "system in the nation. We can change it by investing in housing, downtowns",
    "and our environment to make Vermont an affordable and attractive place to",
    "live. And we can change it by reaching potential Vermonters and showing",
    "most likely to consider moving to Vermont, tell them our story and make it",
    "interested in what Vermont has to offer.",
    "It includes funding for relocation support to really sell Vermont by",
    "community they\u2019re drawn to, in the regions and job sectors that need them",
    "Vermont businesses.",
    "The fact is: every new worker we recruit to live here allows a business, a",
    "Better roads and bridges? More taxpayers.",
    "and allows us to continue funding programs and services that Vermonters",
    "this institution. I believe in each of you and our ability to solve",
    "good, the good that comes from the people of Vermont. And the good that",
    "and division. We can recognize Vermonters call for balance, for civility",
    "If we do, we will make a difference in the lives of Vermonters, and our",
    "Tags: budget address, Phil Scott, Vermont legislature",
    "After tense discussion, power outage stalls vote on contentious unemployment\u2026",
    "Burlington police transformation director under fire for plagiarism",
    "* Burlington police transformation director under fire for plagiarism",
    "* Burlington liquor store gutted by fire to return as Pearl Street Pipe",
    "* Legislators reject tax hike on the wealthy to help solve pension",
    "Purple Heart recipient, CIA staffer, sheep farmer and sugarmaker",
    "After tense discussion, power outage stalls vote on contentious unemployment",
    "Burlington police transformation director under fire for plagiarism",
    "Carol Weston promoted to Efficiency Vermont Director",
    "Alliance for a Better Vermont, elected officials and community leaders call on",
    "Steve Crowley: Time for Vermonters to help create our climate future",
    "After tense discussion, power outage stalls vote on contentious unemployment",
    "(802) 225-6224",
    "www.vtdigger.org",
    "* [ ] Sunday Only (Weekly Wrap)",
    "VTDigger regularly publishes stories about Vermont politics. We cover",
    "state elections, the Vermont Legislature, the governor\u2019s office, state",
    "VTDigger covers criminal justice issues including Vermont legal",
    "issues, Vermont courts and the Vermont Department of Corrections.",
    "VTDigger publishes daily stories on health care. We cover the state\u2019s",
    "hospitals, the state\u2019s psychiatric care system, the Green Mountain",
    "Erin Mansfield is VTDigger\u2019s health care reporter. She can be reached",
    "VTDigger publishes stories about Vermont environmental issues,",
    "VTDigger publishes a wide range of stories about Vermont\u2019s educational",
    "Commentary policy VTDigger.org publishes 12 to 18 commentaries a week"
]

In [12]:
input = ""
for d in data:
    input += d

In [3]:
claim = "\"Every county, including Chittenden, has fewer people available for work.\""

In [15]:
prompt = f"""Carefully evaluate the authenticity of a claim and generate a no more than 50-words argument by reading a reference. Note: If the provided reference is not related to the claim, directly output not related.\nClaim: {claim}\nReference: {data if len(input)<3000 else input[:3000]}"""

In [16]:
prompt

'Carefully evaluate the authenticity of a claim and generate a no more than 50-words argument by reading a reference. Note: If the provided reference is not related to the claim, directly output not related.\nClaim: "Every county, including Chittenden, has fewer people available for work."\nReference: Rigorous nonprofit news for Vermont.* Teachers, high-risk Vermonters eligible for vaccine* \'Virus in Vermont\' seriesGovernor Scott: Mr. President, Madam Speaker, Mr. Chief Justice, membersof the General Assembly, distinguished guests and fellow Vermonters:dollar we spend comes directly from hardworking Vermonters, manygrowing our economy, making Vermont more affordable and protecting theTo put this in perspective, every county except Chittenden has seen adecline in the number of people working. Every county, includingChittenden, has fewer people available for work.and expand, harder for communities to grow and leaves fewer of us to coverAs our working-age population continues to decline

In [10]:
import requests
import json

payload={
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5,
    }
headers={
        "Authorization": f"Bearer {key}",
        "Content-Type":"application/json"
    }
url = "https://api.openai.com/v1/chat/completions"
res = requests.post(url=url, data=json.dumps(payload), headers=headers)
res = json.loads(res.text)
res['choices'][0]['message']['content']

'Not related.'